In [ ]:
import os
from brtdevkit.data import Dataset
from email.mime import image
import itertools
from pathlib import Path
import torch
import clip
from PIL import Image
from torch.utils.data import Dataset as torchDataset, DataLoader
import torchvision.transforms as T
import torch.nn as nn
from skimage.metrics import structural_similarity as ssim
from tqdm import tqdm
from matplotlib import pyplot as plt
from sklearn.cluster import KMeans
import pandas as pd
import numpy as np
import yaml
from pathlib import Path 
from sklearn.decomposition import PCA
torch.cuda.empty_cache()

import imageio

In [ ]:
class Datasetpreparer(torchDataset):    
        def __init__(self, data, preprocess=None, transform=None, device="cuda"):
            self.data = data
            self.transform =  transform 
            self.preprocess = preprocess
            self.device = device 
        def __len__(self):
            return len(self.data)

        def __getitem__(self, idx):
            img_path = self.data.loc[idx, 'image_path']
            img_pil = self.transform(Image.fromarray(imageio.imread(img_path)))
            # Download and open the image        
            img_pil = self.preprocess(img_pil).to(self.device)
            return img_pil, img_path

class ImageSimilarity: 
    def __init__(self, images_full_path, data_base_path, dataset_name='image_list', overwrite=False):
        self.data_base_path = data_base_path
        os.makedirs(self.data_base_path, exist_ok=True)
        self.images_base_path = data_base_path + "/images"
        self.images_full_path = images_full_path
        
        self.image_path_df = None 
        self.transform = T.Compose([
            T.Resize((224,224))])
        
        self.inference_set = None 
        self.embeddings_np = None
        self.image_paths = None 
        self.image_paths_df = None
        self.embeddings_save_name = f"{dataset_name}_embeddings"
        self.embeddings_image_paths = Path(self.data_base_path) / f"{dataset_name}_embeddings_image_paths.csv"
        self.embeddings_save_loc = Path(self.data_base_path) / self.embeddings_save_name
        self.overwrite = overwrite 
        self.model = None 
        self.preprocessor = None 
        self.sorted_scores = None 
        self.sorted_scores_save_loc = data_base_path 
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.load_model()
        
    def prepare_images_path_df(self): 
        self.image_path_df = pd.DataFrame(data=self.images_full_path, columns=['image_path'])
        return self.image_path_df
    
    def prepare_dataloader(self): 
        self.inference_set = Datasetpreparer(data=self.image_path_df, preprocess=self.preprocessor, transform=self.transform, device=self.device)
        inference_loader = DataLoader(self.inference_set, batch_size=64, shuffle=False, num_workers=0, drop_last=True)
        self.total = len(self.image_path_df) / 64
        return inference_loader
 
    def load_model(self): 
        self.model, self.preprocessor = clip.load("ViT-B/32", device=self.device)
        return self.model, self.preprocessor

    def build_embeddings(self): 
        outputs = []
        image_paths = []
        if (os.path.isfile(self.embeddings_save_loc)==True) or (self.overwrite==False): 
            self.embeddings_np, self.image_paths = self.load_embeddings()
            return self.embeddings_np, self.image_paths 
        else:
            inference_loader = self.prepare_dataloader()
            for idx, batch in tqdm(enumerate(inference_loader), total=self.total): 
                batch, paths = batch[0], batch[1]
                with torch.no_grad():
                        batch = batch.to(self.device)
                        image_features = self.model.encode_image(batch)
                        outputs.append(image_features)  
                        image_paths.extend(paths)
            outputs = torch.cat(outputs, dim=0)
            self.image_paths = image_paths 
            self.embeddings_np = outputs.detach().cpu().numpy()
            return self.embeddings_np, self.image_paths
            
    def save_embeddings(self): 
            np.savez(self.embeddings_save_loc, self.embeddings_np) 
            image_paths_df = pd.DataFrame(data=self.image_paths, columns=['image_path'])
            image_paths_df.to_csv(self.embeddings_image_paths, index=False)
            return None 
        
    def load_embeddings(self): 
        embeddings_save_loc = str(self.embeddings_save_loc) +".npz"
        embeddings = np.load(embeddings_save_loc)
        self.embeddings_np = embeddings[embeddings.files[0]]       
        self.image_paths_df= pd.read_csv(self.embeddings_image_paths )
        return self.embeddings_np, self.image_paths_df 

    def get_image_embedding(self, image_path): 
        transformed_image = self.transform(Image.open(image_path))
        img = self.preprocessor(transformed_image).unsqueeze(0).to(self.device)
        embedding = self.model.encode_image(img)
        embeddings_np = embedding.detach().cpu().numpy()
        return embeddings_np
    
    def get_similar_images(self, image_path, calcuate_embeddings=False, images_base_path=None): 
        if images_base_path==None:
            images_base_path = self.images_base_path
        full_image_path = os.path.join(images_base_path, image_path)
        embeddings_np, image_paths_df = self.load_embeddings()
        image_paths = image_paths_df['image_path'].tolist()
        if calcuate_embeddings==True:
            reference_embedding = self.get_image_embedding(full_image_path)
        else:
            embedding_index = image_paths_df[image_paths_df['image_path'] == full_image_path].index[0]
            reference_embedding = embeddings_np[embedding_index, :]
        
        similarity_score = np.dot(reference_embedding, embeddings_np.T)/(np.linalg.norm(reference_embedding)*np.linalg.norm( embeddings_np, axis=1))
        score_df = pd.DataFrame(data=list(similarity_score.T), columns=['similarity_score'])
        score_df['image_path'] = image_paths 
        self.sorted_scores = score_df.sort_values(by='similarity_score', ascending=False)
        image_name = image_path.split("/")[-1].replace(".png","")
        save_loc_sorted_scores = Path(self.sorted_scores_save_loc) / f"{image_name}.csv"
        self.sorted_scores.to_csv(save_loc_sorted_scores, index=False)
        return self.sorted_scores
    
    def similar_images_with_text(self, text_prompt):
        
        embeddings_np, image_paths_df = self.load_embeddings()
        image_paths = image_paths_df['image_path'].tolist()
        text = clip.tokenize(text_prompt).to(self.device)
        text_features = self.model.encode_text(text)
        text_features /= text_features.norm(dim=-1, keepdim=True)
        text_features = text_features.detach().cpu().numpy()
        reference_embedding = text_features
            
        similarity_score = np.dot(reference_embedding, embeddings_np.T)/(np.linalg.norm(reference_embedding)*np.linalg.norm( embeddings_np, axis=1))
        score_df = pd.DataFrame(data=list(similarity_score.T), columns=['similarity_score'])
        score_df['image_path'] = image_paths 
        self.sorted_scores = score_df.sort_values(by='similarity_score', ascending=False)
        image_name = text_prompt.replace(" ","_")
        save_loc_sorted_scores = Path(self.sorted_scores_save_loc) / f"{image_name}.csv"
        self.sorted_scores.to_csv(save_loc_sorted_scores, index=False)
        return self.sorted_scores
        
        
    def plot_image_grid(self, image_paths_df, tail=0  ,nrows=5, ncols=2):
        
        # Assume image_paths is your list of image file paths
        if tail > 0:
            image_paths_df = image_paths_df.tail(tail)
        image_paths = image_paths_df['image_path'].tolist()
        n_rows = min(nrows, int(image_paths_df.shape[0] // 2 ))
        fig, axes = plt.subplots(n_rows, ncols, figsize=(10, nrows*5))
        for i, ax in enumerate(axes.flatten()):
            if i < len(image_paths)-1:
                image_path = image_paths[i]
                title = image_path.split("/")[-1]
                img = Image.open(image_path)
                ax.imshow(img)
                ax.set_title(f'{title}  indx: {i}', fontsize=6)
            ax.axis('off')

        plt.tight_layout()
        plt.show()


    def extract_embeddings(self): 
        self.prepare_images_path_df()
        self.prepare_dataloader()
        self.overwrite = True
        self.build_embeddings()
        self.save_embeddings()
        return None 

In [ ]:
dsetname ='mannequin_in_dust_v3'
aletheia_ds = Dataset.retrieve(name=dsetname)
aletheia_df = aletheia_ds.to_dataframe()
dataset_save_dir = os.environ['DATASET_PATH'] + "/" + dsetname
if not os.path.exists(dataset_save_dir):
    os.makedirs(name=dataset_save_dir)
    aletheia_ds.download(dataset_save_dir)

In [ ]:
def get_images(save_dir, df):
    save_dir = Path(save_dir)
    dirs = [save_dir / 'images' / d for d in df.id]#[df['label_human']].id]
    image_paths = []
    for i, dir in enumerate(dirs):
        if i % 10 == 0:
            print(i / len(str(dir)))
        for fname in os.listdir(dir):
            if 'debayeredrgb' in fname:
                image_paths.append(str(dir / fname))
    return image_paths
image_paths = get_images(dataset_save_dir, aletheia_df)
# save_file = Path(dataset_save_dir) / "image_ids.npy"
# image_paths = np.load(save_file).tolist()

In [ ]:
sim = ImageSimilarity(images_full_path=image_paths, data_base_path=dataset_save_dir, dataset_name=dsetname, overwrite=False)

In [ ]:
sim.extract_embeddings()
embeddings_np, paths_df = sim.load_embeddings()

In [ ]:
n_images_final = 1000
kmeans = KMeans(n_clusters=n_images_final, random_state=0, n_init="auto")
kmeans.fit(embeddings_np)
final_paths: list[None] = [None for _ in range(n_images_final)]

In [ ]:
final_paths = [None for _ in range(n_images_final)]

import random
order = list(enumerate(kmeans.labels_))
random.shuffle(order)

for i, l in order:
    if final_paths[l] == None:
        final_paths[l] = paths_df.image_path.iloc[i]

In [ ]:
imids = [p.split('_')[-1][:-4] for p in final_paths]
print(len(imids))

In [ ]:
# from aletheia_dataset_creator.dataset_tools.aletheia_dataset_helpers import imageids_to_dataset
# desc = f"Mannequins standing in dust, diversified. ({len(imids)} images)"
# imageids_to_dataset(image_ids=imids, dataset_name="mannequin_in_dust_v0_diverse", dataset_description=desc, dataset_kind=Dataset.KIND_ANNOTATION, production_dataset=False)
